<a href="https://colab.research.google.com/github/Seowoo-Jung/ESAA/blob/main/9.%20%EC%B6%94%EC%B2%9C%EC%8B%9C%EC%8A%A4%ED%85%9C(1)___5%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 추천시스템(1):5장 

## 콘텐츠 기반 필터링 실습 - TMDB 5000 영화 데이터 세트

### p.579 ~ 580

>> 콘텐츠 기반 필터링: 사용자가 특정한 아이템을 선호하는 경우 그 아이템과 비슷한 콘텐츠를 가진 다른 아이템 추천

--> 이 실습의 경우 장르 속성을 기반으로 만든다.



In [ ]:
import pandas as pd
import numpy as np
import warnings; warnings.filterwarnings('ignore')

movies = pd.read_csv("/content/tmdb_5000_movies.csv")
print(movies.shape)
movies.head(1)

(4803, 20)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, ...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century Fox Film Corporatio...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""GB"", ""name"": ""United ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [ ]:
## 콘텐츠 기반 필터링 추천 분석에 필요한 내용만 추출
movies_df = movies [['id', 'title','genres','vote_average', 'vote_count', 'popularity', 'keywords','overview']]

In [ ]:
# genres, keywords의 경우 여러개의 장르를 표현하기 위해 딕셔너리에 표현되어있음!--> 가공해야 정보를 추출할 수 있음!
# 어떤게 있는지 확인
pd.set_option('max_colwidth',100)
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id"": 3386, ""name"": ""sp..."


In [ ]:
from ast import literal_eval #문자열을 list[dict1, dict2] 로 분해할 수 있음. # 리스트가 문자열이 었는데 리스트 객체 로 만들어줌
movies_df['genres'] = movies_df['genres'].apply(literal_eval) #보기
movies_df['keywords'] = movies_df['keywords'].apply(literal_eval)

In [ ]:
type(movies_df['genres'][0][1])
movies_df['genres'][0][1] # 리스트 내부에 딕셔너리로 구성된 객체를 가짐. 

'Adventure'

In [ ]:
# 장르명만 추출!
movies_df['genres'] = movies_df['genres'].apply(lambda x : [ y['name'] for y in x])
movies_df['keywords'] = movies_df['keywords'].apply(lambda x : [ y['name'] for y in x])
movies_df[['genres','keywords']][:1]

,genres,keywords
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colony, society, space travel, futuristic, romance, spa..."


## 장르 콘텐츠 유사도 측정

- genres를 count 기반으로 벡터 변환한다.
- 벡터 변환한 데이터를 코사인 유사도를 이용하여 비교한다. 이를 위해 데이터 세트의 레코드별로 보사인 유사도값을 가진 객체 생성
- 장르 유사도보다 높은 영화 중 평점 높은순으로 추천한다.

In [ ]:
# CountVectorizer이용을 위해서 공백 문자로 word 단위가 구분되는 문자열로 변환. (list->문자열)
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x : (' ').join(x)) 
movies_df['genres_literal'][0]

'Action Adventure Fantasy Science Fiction'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat=count_vect.fit_transform(movies_df['genres_literal'])
print(genre_mat.shape)

(4803, 276)


In [ ]:
# 코사인 유사도를 적용하여 행렬 형태로 반환
from sklearn.metrics.pairwise import cosine_similarity

genre_sim=cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:1]) #여기서 유사도가 높은 순서대로 정리

(4803, 4803)
[[1.         0.59628479 0.4472136  ... 0.         0.         0.        ]]


In [ ]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1] # 각 행에 있는것들 중 유사도가 높은 순으로 정렬
# argsort는 정렬된 비교 대상의 index를 반환해준다.
print(genre_sim_sorted_ind[:1]) # index 이용하여 특정 레코드 추출할 수 있음.

[[   0 3494  813 ... 3038 3037 2401]]


### 장르 콘텐츠 필터링을 이용한 영화 추천



In [ ]:
## 함수를 만들어서 추천영화 정보를 갖는 dataframe 반환.
def find_sim_movie(df, sorted_ind, title_name, top_n=10): #df:movide_df, sorted_ind:코사인 유사도 인덱스, 
#title_name: 추천기준이 되는 영화제목, top_n: 추천영화건수
  # 인자로 입력된 movie_df Dataframe에서 'title' 칼럼이 입력된 title_name 값인 DataFrame추출
  title_movie = df[df['title'] == title_name]

  # title_named을 가진 DataFrame의 index 객체를 ndarray로 반환(원소 1개)하고
  # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n개의 index 추출
  title_index = title_movie.index.values
  similar_indexes = sorted_ind[title_index, :(top_n)]

  # 추출된 top_n index 출력. top_n index는 2차원 데이터임.
  # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
  print(similar_indexes)
  similar_indexes = similar_indexes.reshape(-1)

  return df.iloc[similar_indexes]

In [ ]:
similar_movies=find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather',10)
similar_movies[['title','vote_average']] # 대부를 좋아하는 고객에게 추천하기에는 이해하기 어려운 영화도 있고 평점이 낮은 영화 존재

[3337]
[[2731 1243 3636 1946 2640 4065 1847 4217  883 3866]]


,title,vote_average
2731,The Godfather: Part II,8.3
1243,Mean Streets,7.2
3636,Light Sleeper,5.7
1946,The Bad Lieutenant: Port of Call - New Orleans,6.0
2640,Things to Do in Denver When You're Dead,6.7
4065,Mi America,0.0
1847,GoodFellas,8.2
4217,Kids,6.8
883,Catch Me If You Can,7.7
3866,City of God,8.1


In [ ]:
# 문제점 개선을 위해 더 많이 뽑아서 평점 순서대로 필터링
# 평점(vote_average)만 보면 소수의 관객이 특정 영화에 높은 평점을 부여해 왜곡이 있을 수 있으므로 vote_count까지 고려
movies_df[['title','vote_average','vote_count']].sort_values('vote_average', ascending=False)[:10]

,title,vote_average,vote_count
3519,Stiff Upper Lips,10.0,1
4247,Me You and Five Bucks,10.0,2
4045,"Dancer, Texas Pop. 81",10.0,1
4662,Little Big Top,10.0,1
3992,Sardaarji,9.5,2
2386,One Man's Hero,9.3,2
2970,There Goes My Baby,8.5,2
1881,The Shawshank Redemption,8.5,8205
2796,The Prisoner of Zenda,8.4,11
3337,The Godfather,8.4,5893


- 위 결과를 보면, 앞에 7개 영화들 같은 경우 적은 관객이 높은 평점을 주어 평점이 상대적으로 높아 영화 평점을 다른 방식으로 산출한다. : 가중 평점 이용!

weighted Rating: (v/(v+m))* R + (m/(v+m)) * C

v: 개별 영화에 평점을 투표한 횟수 : 'vote_count'

m: 평점을 부여하기 위한 최소 투표 횟수 : 가중치 조절: m이 높으면 평점 투표횟수가 많은 영화에 더 큰 가중치!(전체 투표 횟수의 상위 60%를 기준으로 한다) :v가 크면 C보다 R에 더 가중치가 더해짐

R: 개별 영화에 대한 평균 평점 :'vote_average'

C: 전체 영화에 대한 평균 평점 : 'vote_average'의 평균

In [ ]:
C=movies_df['vote_average'].mean()
m=movies_df['vote_count'].quantile(0.6)
print('C:', round(C, 3), 'm: ', round(m,3))

C: 6.092 m:  370.2


In [ ]:
# weighted mean을 구하는 함수를 생성하고 새로운 평점 정보('vote_weighted') 추가
percentile=0.6
m=movies['vote_count'].quantile(percentile)
C=movies['vote_average'].mean()

def weighted_vote_average(record):
  v=record['vote_count']
  R=record['vote_average']

  return ((v/(v+m))*R) + ( (m/(m+v))*C)

movies_df['weighted_vote']=movies_df.apply(weighted_vote_average, axis=1)

In [ ]:
# 가중 평점 기준으로 출력
movies_df[['title', 'vote_average', 'weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.396052,8205
3337,The Godfather,8.4,8.263591,5893
662,Fight Club,8.3,8.216455,9413
3232,Pulp Fiction,8.3,8.207102,8428
65,The Dark Knight,8.2,8.136930,12002
1818,Schindler's List,8.3,8.126069,4329
3865,Whiplash,8.3,8.123248,4254
809,Forrest Gump,8.2,8.105954,7927
2294,Spirited Away,8.3,8.105867,3840
2731,The Godfather: Part II,8.3,8.079586,3338


In [ ]:
## 개인에게 맞는 수정된 영화 추천시스템
# 기존 함수보다 더 많은 영화 추천 후보군을 뽑고 weighted_vote 기준으로 거른다.
def find_sim_movie(df, sorted_ind, title_name, top_n=10):
  title_movie=df[df['title'] == title_name]
  title_index = title_movie.index.values

  # top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
  similar_indexes = sorted_ind[title_index, :(top_n*2)]
  similar_indexes = similar_indexes.reshape(-1)
  # 기준 영화 인덱스는 제외
  similar_indexes = similar_indexes[similar_indexes != title_index]

  # top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
  return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title', 'vote_average', 'weighted_vote']]

,title,vote_average,weighted_vote
2731,The Godfather: Part II,8.3,8.079586
1847,GoodFellas,8.2,7.976937
3866,City of God,8.1,7.759693
1663,Once Upon a Time in America,8.2,7.657811
883,Catch Me If You Can,7.7,7.557097
281,American Gangster,7.4,7.141396
4041,This Is England,7.4,6.739664
1149,American Hustle,6.8,6.717525
1243,Mean Streets,7.2,6.626569
2839,Rounders,6.9,6.530427
